# Measure Astigmatism on AuxTel --- SITCOM-627

This notebook contains the execution for the measuring of astigmatism described in https://jira.lsstcorp.org/browse/SITCOM-627.
                
The notebook is organized in 6 sections:

    1.1 Conditions assessment
    1.2 Setup notebook
    1.3 Declare target and filter
    1.4 Slew to target object.
    1.5 Check target star field, signal level and declare exposure time. 
    1.6 Perform the data acquisition sequence. 
    


Goal: Directly measure image degradation due to astigmatism on AuxTel by sweeping through focus.

Idea: If we sweep through focus we will be able to measure image degradation due to astigmatism by (a) comparing the FWHM of each individual axis at its best focus to the FWHM of the circle of least confusion and (b) measuring the distance between the best focus of each axis (these measurements should agree, and hopefully also agree with Zernikes).

Observing sequence:

    On a night with decent seeing (< 1.5 arcsec), focus on star field with medium to low density of stars.
    Sweep through focus by moving M2 in steps of 0.02 mm from -0.1 mm to +0.1 mm (the idea is to move the focal plane in steps of ~1mm). Take 2 x 30s exposures at each position with the red filter in. Then take 2x 30s exposures with the holo disperser and no blocking filter in. 
    Slew to a nearby bright star and perform CWFS/WEP (or this can be done as step one)

----
## Assess that conditions meet criteria
This test should be performed when the seeing is "decent", under 1.5 arcsec. 

Preferably, run this execution later in the night, when dome seeing has settled down and seeing is more stable. Confirm that the temperatures in and outside the dome are within 1 degree. 

-----
## Setup

### Import libraries

In [1]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

/home/isotuela/auto-op-env-packages/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_acquire_and_take_sequence.py:49: UserWarning: Cannot import required libraries. Script will not work.
  warnings.warn("Cannot import required libraries. Script will not work.")
/home/isotuela/auto-op-env-packages/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_cwfs_align.py:47: UserWarning: Cannot import required libraries. Script will not work.
  warnings.warn("Cannot import required libraries. Script will not work.")


### Setting up logger

In [2]:
logger = logging.getLogger("SITCOM-627")
logger.level = logging.DEBUG

### Getting unique index for script

In [3]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [4]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [5]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [6]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [ ]:
script.log.info(f'START -- SITCOM-627 Astigmatism test -- at {Time.now()}')

----
## Target and filter

### Declare target: Choose target - Star field needs to be low to medium density. 
ATCS will query SIMBAD for a `maglim_target` 8.0 maximum magnitude target around `az_target` 182 and `el_target` 65 with a `radius_target` of 2 degrees

Modify cell below to choose different target coordinates. 

In [ ]:
az_target = 182
el_target = 65
maglim_target = 8.0
radius_target = 2

In [ ]:
target = await script.atcs.find_target(az=az_target, el=el_target, mag_limit=maglim_target, radius = radius_target)
logger.info(f'Target with magnitude limit {maglim_target} is {target}')

### Declare filter - Set up with reddest band in filter wheel (to get best image quality) and no disperser

In [7]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'Available filters are: {inst_setup[0]},\n Gratings are: {inst_setup[1]}')

In [ ]:
filter_to_use = 'OG550_65mm_1'

------
## Slew to target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

### ATAOS corrections

ATAOS corrections must be enabled for this test. In the cell below, ATAOS corrections will be enabled, in case they were not. 

In [ ]:
corrections_enabled = await script.atcs.rem.ataos.evt_correctionEnabled.aget()
if not (corrections_enabled.m1 and corrections_enabled.hexapod and corrections_enabled.atspectrograph):
    cmd = await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
    logger.info(f'ATAOS corrections enabled: {cmd.result}')
else:
    logger.info(f'ATAOS corrections already enabled')

-------
## Check star field and signal levels. Declare exposure time

Take an image to check the star field. It should have a low to medium density of stars with a range of magnitudes will be important. 
To change the field, re-run this notebook from section 1.3 with different coordinates. 

In [ ]:
acq_exposure = await latiss.take_acq(
        exptime=30, n=1, filter=filter_to_use, grating='empty_1', reason='Acquisition', program="SITCOM-627")
logger.info(f'Acquisition exposure is {acq_exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Declare exposure time

Once happy with the chosen star field, declare the exposure time for the data acquisition loop. It should be 30 seconds. 

In [ ]:
exp_time = 30

------
## CWFS
A CWFS is recommended just before the test to ensure the system is in focus. The same target will be used to perform the CWFS.  

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_use, 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "program" : "SITCOM-627"})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

----
## Data Acquisition Sequence: 
Sweep through focus by moving M2 in steps of 0.02 mm from -0.1 mm to +0.1 mm 

###  Define sweep 
Acquire data in steps of `z_offset_step` in the range of `z_offset_start` to `z_offset_end` relative to the current focus position.

In [8]:
z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start

steps = np.arange(z_offset_start, z_offset_end + 0.01, z_offset_step)

logger.info(f'The hexapod will be moved the following z offset steps \n {steps} [mm]. \n To modify the sweep size or range, change the parameters above and run this cell again')

### Original focus offset

In [9]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Original focus offset is {original_focus_offset}')

### Data Acquisition Loop
This loop will take n_images=2 with the chosen `filter_to_use` and no disperser in each of the defined hexapod position. 

#### Filter and no grating

In [ ]:
data = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
        
    exposure = await latiss.take_engtest(
        exptime=exp_time, n=2, filter=filter_to_use, grating='empty_1', reason='Focus_sweep', program="SITCOM-627")
    data.append(exposure)
    script.log.info(f'Total z offset = {step:.3f} mm -- Image expId = {exposure}')
    
    logger.info(f'Current focus offset is \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

In [ ]:
logger.info(f'Images with filter {filter_to_use} are {data}')

##### Check that hexapod is back to the original focus

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus offset is {current_focus_offset}'

In [ ]:
diff = current_focus_offset.userApplied - original_focus_offset.userApplied
print(f'Difference between current and original user Applied focus is {diff:0.2f} mm')

#### No blocking filter and holo disperser on

Now, run the same focus sweep with no blocking filter and the holo disperser on. 

In [ ]:
data_holo = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
        
    exposure = await latiss.take_engtest(
        exptime=exp_time, n=2, filter='empty_1', grating='holo4_003', reason='Focus_sweep', program="SITCOM-627")
    data_holo.append(exposure)
    script.log.info(f'Total z offset = {step:.3f} mm -- Image expId = {exposure}')
    
    logger.info(f'Current focus offset is \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

In [ ]:
logger.info(f'Images with no blocking filter and the holo grating are {data_holo}')

##### Check that hexapod is back to the original focus

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus offset is {current_focus_offset}'

In [ ]:
diff = current_focus_offset.userApplied - original_focus_offset.userApplied
print(f'Difference between current and original user Applied focus is {diff:0.2f} mm')

### Stop tracking

In [ ]:
await atcs.stop_tracking()

### Write end info into EFD

In [ ]:
script.log.info(f'END -- SITCOM-627 Astigmatism test -- at {Time.now()}')